In [1]:
#pip install tmdbsimple
import pandas as pd
import numpy as np
from numpy.random import shuffle
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
import tmdbsimple as tmdb
tmdb.API_KEY = 'abcd9827ce3fb2298891370ff0cd0e58'

In [23]:
def takeInfos(id):
    pessoas = []
    keywords = []
    sTitles = []
    sPopularity = []
    sMedia = []

    movie = tmdb.Movies(id)

    response = movie.info()
    popularity = round(movie.popularity)
    media = round(movie.vote_average)
    title = movie.title
    for ator in movie.credits()['cast']: pessoas.append(ator['name'])
    for crew in movie.credits()['crew']: pessoas.append(crew['name'])
    for keyword in movie.keywords()['keywords']: keywords.append(keyword['name'])
    for similar in movie.similar_movies()['results']: 
        sTitles.append(similar['title'])
        sPopularity.append(round(similar["popularity"]))
        sMedia.append(round(similar["vote_average"]))
    
    return title,popularity,media,pessoas,keywords,sTitles,sPopularity,sMedia

def NormalizeCat(X):
    global perda
    i = 0
    n = len(X)
    limit = 300
    if len(X)>limit:perda+=1
    while(len(X)<limit):
        if i == n: i = 0
        X.append(X[i])
        i+=1
    while(len(X)>limit):
        X.pop()
    return X
        

In [3]:
class Encoder:
    def __init__(self,lista):
        self.Codex = []
        self.atualIndex = 1
        self.fit(lista)
        
    def fit(self,lista):
        for item in lista:
            onList = False
            for code in self.Codex:
                if item == code[0]: onList = True
            if not onList:
                self.Codex.append([item,self.atualIndex])
                self.atualIndex+=1
    
    def transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[0]: resposta.append(code[1])
        return resposta
    
    def fit_transform(self,lista):
        self.fit(lista)
        return self.transform(lista)
    
    def inverse_transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[1]: resposta.append(code[0])
        if len(lista)==1: return resposta[0]
        return resposta
    
    def show(self):
        print(self.Codex)
    
    def Feed(self,id):
        title,popularity,media,pessoas,keywords,sTitles,sPopularity,sMedia = takeInfos(id)
        self.fit([title])
        self.fit(sTitles)
        self.fit(pessoas)
        self.fit(keywords)
        Ms = []
        Pss = []
        for M in sMedia: Ms.append("M"+str(M))
        for Ps in sPopularity: Pss.append("Ps"+str(Ps))

        X = self.fit_transform(["P"+str(popularity)])
        for i in self.fit_transform(Pss): X.append(i)
        for i in self.transform(pessoas): X.append(i)
        for i in self.transform(Ms): X.append(i)
        for i in self.transform(keywords): X.append(i)
        y = [media]
        X = [NormalizeCat(X)]
        return X,y

In [4]:
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = pd.read_csv("tmdb_5000_movies.csv")

In [5]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
Naive = MultinomialNB()

In [25]:
print(perda)

0


In [9]:
#NÃO RODAR ISSO SE NÃO FOR MUUUITO NECESSARIO
Codex = Encoder(['M0',"M1","M2",'M3','M4','M5','M6','M7','M8','M9','M10'])
perda = 0
X,y = Codex.Feed(603)
if not (len(X[0])>300) :Naive.fit(X,y)
else: perda+=1

C:\Users\pheli\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


In [24]:
n = 30
i = 0
IDlist = IDlist = list(movies["id"])
shuffle(IDlist)
for idd in IDlist:  
    X,y = Codex.Feed(idd)
    if not (len(X[0])>300):Naive.fit(X,y)
    else: perda+=1
    i += 1
    print(str(i)+"/"+str(n))
    if i == n:break

C:\Users\pheli\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


1/30
2/30
3/30
4/30
5/30
6/30
7/30
8/30
9/30
10/30
11/30
12/30
13/30
14/30
15/30
16/30
17/30
18/30
19/30
20/30
21/30
22/30
23/30
24/30
25/30
26/30
27/30
28/30
29/30
30/30


In [26]:
i = 0
n = 10
s8 = 0
n8 = 0
e1 = 0
em1 = 0
a = 0

IDlist = IDlist = list(movies["id"])
shuffle(IDlist)
for idd in IDlist: 
    X,y = Codex.Feed(idd)
    saida = Naive.predict(X)
    
    if saida != 8: n8 += 1
    else: s8 += 1
    
    if saida == y: a += 1
    elif (saida == y[0]+1 or saida == y[0]-1): e1 += 1
    else: em1 +=1

    i += 1
    print(str(i)+"/"+str(n))
    if i == n:break
        
print("\n a: {0}".format(a))
print("e1: {0}".format(e1))
print("em1: {0}".format(em1))
print("\n n8: {0}".format(n8))
print(" s8: {0}".format(s8))

1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10

 a: 5
e1: 4
em1: 1

 n8: 10
 s8: 0


In [ ]:
search = tmdb.Search()
response = search.movie(query='The Man Who Invented')
for s in search.results: print(s['title'], s['id'], s['release_date'], s['popularity'], round(s['vote_average']))